#### Transformers
##### Использование траснформеров для саммаризации текстов

In [2]:
!pip install sentencepiece

In [5]:
import transformers
import datasets
import time

In [6]:
from datasets import load_dataset
ds_gazeta = load_dataset('IlyaGusev/gazeta', revision="v1.0")

No config specified, defaulting to: gazeta_dataset/default
Reusing dataset gazeta_dataset (/home/medic/.cache/huggingface/datasets/IlyaGusev___gazeta_dataset/default/1.0.0/ef9349c3c0f3112ca4036520d76c4bc1b8a79d30bc29643c6cae5a094d44e457)


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
ds_gazeta['train'][0]

{'text': '«По итогам 2011 года чистый отток может составить примерно $80 млрд, в следующем году — около $20 млрд. При этом мы ожидаем, что со второго полугодия 2012 года начнется приток капитала», — заявил «Интерфаксу» замминистра экономического развития Андрей Клепач. Официальные прогнозы по выводу капитала из России становятся все пессимистичными: еще летом власти полагали, что из страны уйдет не более $35 млрд, в сентябре Минэкономразвития назвал цифру $50 млрд, в начале ноября Центробанк пересмотрел оценку до $70 млрд. Очередное изменение прогноза было ожидаемо: по расчетам Центробанка , за январь — октябрь чистый отток капитала достиг $64 млрд, причем в последние месяцы он ускорился: в сентябре он составил $14 млрд, в октябре — $13 млрд против среднего ежемесячного оттока в $6—8 млрд в первом полугодии. «После октябрьских данных Минэкономразвития вынуждено было изменить оценку, настаивать на $70 млрд означало ожидать серьезного замедления оттока капитала на непонятно каких причина

In [8]:
train_data = ds_gazeta['train']
val_data = ds_gazeta['validation']
test_data = ds_gazeta['test']

In [9]:
from transformers import AutoTokenizer, MBartForConditionalGeneration #AutoModel

model_name = "IlyaGusev/mbart_ru_sum_gazeta"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

In [10]:
train_data[133]['text']

'Портфель необеспеченных кредитов населению РФ вырос на 65%, до 4,1 трлн рублей, оценили аналитики агентства «Эксперт РА», подготовившие исследование рынка розничного кредитования с 1 июля 2011 года по июль 2012-го. Это более чем вдвое превышает ипотечный портфель банков (1,6 трлн рублей) и сопоставимо с портфелем кредитов малому и среднему бизнесу (4,2 трлн рублей). Темпы кредитования существенно опережают динамику привлечения вкладов. Ранее зампред правления ВТБ24 Екатерина Петелина сообщила, что в 2012 году в целом по сектору кредитование населения вырастет на 37–38%, а привлеченные средства физических лиц — на 18–19%. Это, по ее мнению, связано с эффектом отложенного спроса, а также ростом активности банков, ориентирующихся на массовый сегмент. В отчете по банковскому сектору за август агентство Fitch также отмечало повышенный интерес банков к рознице. «Это привело к усилению конкуренции и, в некоторых случаях, понижению стандартов расчета страховых рисков, — предупредили аналитики

In [11]:
%%time
input_ids = tokenizer.prepare_seq2seq_batch(
            [train_data[0]['text']],
            src_lang='ru_XX',
            return_tensors='pt',
            padding='max_length',
            truncation=True,
            max_length=600
                 )['input_ids']

output_ids = model.generate(
             input_ids=input_ids,
             max_length=162,
             no_repeat_ngram_size=3,
             num_beams=5,
             top_k=0)[0]

summary = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

/home/medic/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3524: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
2022-12-05 20:19:15.769432: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object fi

CPU times: user 2min 2s, sys: 2.91 s, total: 2min 5s
Wall time: 46.7 s


In [12]:
print(summary)

Минэкономразвития прогнозирует замедление оттока капитала из России в 2011 году до $80 млрд. Это один из самых высоких за последние 20 лет. Больше ушло лишь в 2008 году на фоне разрастания финансового кризиса и российско-грузинской войны — $133,7 млрд.


In [13]:
%%time
input_ids = tokenizer.prepare_seq2seq_batch(
            [train_data[10]['text']],
            src_lang='ru_XX',
            return_tensors='pt',
            padding='max_length',
            truncation=True,
            max_length=600
                 )['input_ids']

output_ids = model.generate(
             input_ids=input_ids,
             max_length=162,
             no_repeat_ngram_size=3,
             num_beams=5,
             top_k=0)[0]

summary = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

/home/medic/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3524: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


CPU times: user 3min 12s, sys: 2.13 s, total: 3min 14s
Wall time: 1min 48s


In [20]:
def get_summary(article):
    summary = ""
    input_ids = tokenizer.prepare_seq2seq_batch(
            [article],
            src_lang='ru_XX',
            return_tensors='pt',
            padding='max_length',
            truncation=True,
            max_length=600
                 )['input_ids']

    output_ids = model.generate(
             input_ids=input_ids,
             max_length=162,
             no_repeat_ngram_size=3,
             num_beams=5,
             top_k=0)[0]
    
    summary = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    
    return summary
    

In [40]:
for_title = []
for item in train_data[:15]['text']:
    summary = get_summary(item)
    print("-" * 128)
    print(f"Исходный текст: {item}")
    print("*" * 64)
    print(f"Summary of text: {summary}")   
    for_title.append(summary)
  

/home/medic/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3524: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


--------------------------------------------------------------------------------------------------------------------------------
Исходный текст: «По итогам 2011 года чистый отток может составить примерно $80 млрд, в следующем году — около $20 млрд. При этом мы ожидаем, что со второго полугодия 2012 года начнется приток капитала», — заявил «Интерфаксу» замминистра экономического развития Андрей Клепач. Официальные прогнозы по выводу капитала из России становятся все пессимистичными: еще летом власти полагали, что из страны уйдет не более $35 млрд, в сентябре Минэкономразвития назвал цифру $50 млрд, в начале ноября Центробанк пересмотрел оценку до $70 млрд. Очередное изменение прогноза было ожидаемо: по расчетам Центробанка , за январь — октябрь чистый отток капитала достиг $64 млрд, причем в последние месяцы он ускорился: в сентябре он составил $14 млрд, в октябре — $13 млрд против среднего ежемесячного оттока в $6—8 млрд в первом полугодии. «После октябрьских данных Минэкономразвития в

--------------------------------------------------------------------------------------------------------------------------------
Исходный текст: Как повлияло вступление в ВТО на конкурентноспособность российской промышленности, какие угрозы и преимущества несет членство в организации и как России правильно играть по ее правилам, чтобы выграть, обсудили участники «круглого стола» «Вступление России в ВТО и повышение конкурентоспособности российской промышленности» в рамках петербургского форума. 22 августа 2012 года Россия стала полноправным членом ВТО, переговоры о вступлении в которую длились 18 лет. Отношение участников дискуссии к первым итогам членства в ВТО оказалось неоднозначным. Больше всего возражений по отношению к вступлению в ВТО было у главы и основного акционера алюминиевой компании «Русал» Олега Дерипаски. «ВТО — это бассейн с акулами, мы это почувствуем через три-четыре года», — объявил Дерипаска , предрекающий спад производства внутри России и рост безработицы до 12%. 

--------------------------------------------------------------------------------------------------------------------------------
Исходный текст: В Кремле подтвердили факт сбора биологичекого материала россиян некоторыми эмиссарами и неправительственными организациями. По словам пресс-секретаря президента Дмитрия Песков а, такая информация поступила от спецслужб. «Эта информация по линии специальных служб России. Действительно, некоторые эмиссары ведут такую деятельность, представители неправительственных организаций и других органов. Такие случаи были зафиксированы, и у специальных служб, естественно, у президента эта информация имеется», — объяснил Песков. Накануне Владимир Путин сообщил, что «некие иностранные силы» занимаются сбором биоматериала представителей различных российских этносов. Так глава государства прокомментировал слова директора Российского общественного института избирательного права (РОИИП) Игоря Борисова в ходе заседания Совета при президенте по правам человека. Бо

--------------------------------------------------------------------------------------------------------------------------------
Исходный текст: Главный тренер астанинского «Барыса» Андрей Хомутов после поражения от «Динамо» (1:3) в матче регулярного чемпионата КХЛ посетовал на многочисленный травмы и усталость. «Поздравляю «Динамо» с победой, — сказал бывший наставник бело-голубых. — Для нас это был тяжелый матч. Мы выглядели уставшими, поэтому у нас не получилась игра в атаке и особенно в большинстве. Да и оборона хромает. Эти два провала, когда пропустили шайбы, полностью сломали нашу игру. Конечно, потеря четырех ключевых защитников не может не сказаться на результате и качестве игры. Что ж, будем работать. — Нынешнее «Динамо» сильно отличается о от прошлогоднего, которое вы тренировали? — Это не «Динамо», а ХК МВД. Таково мое мнение. Все-таки практически весь состав ХК МВД играет здесь. Теперь это ОХК «Динамо». — А ОХК «Динамо» сильнее той команды, которую вы возглавляли? — Сезон 

--------------------------------------------------------------------------------------------------------------------------------
Исходный текст: Заключенный башкирской исправительной колонии № 2 в городе Салават, протестуя против угроз со стороны администрации, был вынужден семь раз собственноручно вгонять в свое тело куски арматуры и гвозди. Заявление осужденного в Генеральную прокуратуру и видеозапись с его рассказом об угрозах в свой адрес координатор проекта Gulagu.net Дмитрий Пронин предоставил в распоряжение «Газеты.Ru». «Нахожусь в критическом состоянии... внутри груди инородное тело, которое не удаляют по причине того, что мне делали три операции под общим наркозом и сердце может не выдержать,» — с этих слов начинается жалоба Александра Каменко, отбывающего наказание в ИК-2 Республики Башкортастан. Вогнать в грудную клетку кусок металла, как утверждает заключенный, его вынудили избиения и угрозы изнасилования со стороны сотрудников администрации и приближенных к ней заключенных

--------------------------------------------------------------------------------------------------------------------------------
Исходный текст: Одесский областной совет в среду на внеочередной сессии подтвердил статус русского языка как регионального в Одесской области. Аналогичное решение будет принято завтра на сессии Донецкого облсовета. Верховный совет Автономной республики Крым, также экстренно собравшийся по «языковому вопросу», поручил республиканскому совмину разработать программу внедрения региональных языков. В Крыму кроме русского особый статус должен получить также крымско-татарский язык. В ходе последней переписи населения в 2001 году 11,4% крымчан заявили, что считают крымско-татарский своим родным, а это превышает минимальную планку в 10% для придания языку нового статуса. Закон об основах языковой политики был принят Верховной радой в начале июля и подписан президентом Виктором Януковичем в августе. Этот документ фактически приравнивает 18 языков национальных меньшинст

--------------------------------------------------------------------------------------------------------------------------------
Исходный текст: В пятницу, 27 мая, в столичном регионе давление повысится до 749 мм рт. ст. Как пояснили «Газете.Ru» в центре «Фобос» , из-за этого солнце сильнее прогреет воздух: днем ожидается +22…+27°С (ночью +9…+14°С). Атмосфера станет неустойчивой, а это, в свою очередь, вызовет короткие дожди и грозы — преимущественно днем. Из-за духоты, которая всегда возникает перед грозой, жара будет переноситься еще хуже, так что всем страдающим от сердечно-сосудистых заболеваний или болезней органов дыхания стоит быть втройне осторожнее. Ветер ночью переменных направлений, слабый, днем восточный, северо-восточный, 3–8 м/с, при грозе — порывистый. Последний уикенд весны поддержит погодный тренд: синоптическая ситуация в Москве существенно не изменится. Оба дня переменная облачность, днем местами кратковременный дождь, не исключена гроза. Давление высокое — 751 мм рт